In [ ]:
import numpy as np
from numpy.ma import masked_array

import pandas as pd

import time
from tqdm import tqdm

import copy

import math

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA


raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"
dist = 5
days = 21

# insert main dir
main_dir = ""

In [ ]:
x_max_t = data_tm['x'].max()
x_min_t = data_tm['x'].min()
y_max_t = data_tm['y'].max()
y_min_t = data_tm['y'].min()
print(x_min_t, x_max_t, y_min_t, y_max_t, (-x_min_t+x_max_t)/200, (-y_min_t+y_max_t)/200)

part_x = 13200
x_min = x_min_t + part_x
x_max = x_max_t - part_x
recs_x = (-x_min+x_max)/200
print(x_min, x_max, recs_x)

part_y = 8400
y_min = y_min_t + part_y
y_max = y_max_t - part_y
recs_y = (-y_min+y_max)/200
print(y_min, y_max, recs_y)

print(recs_x * recs_y)

In [ ]:
# Select sub Prague rectangles and sort by date - complet data
data_dir = "%s/data/%s/tmdata_complet_%d_%d_%s_%s_%s.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data = pd.read_csv(data_dir)
print(data.shape)

x_min = -744800
x_max = -734800
y_min = -1051600
y_max = -1041600

data = data.loc[lambda data: (data.praha == 1) & (data.x200 >= x_min) & (data.x200 <= x_max) & (data.y200 >= y_min) & (data.y200 <= y_max), :]
data.sort_values('date',inplace=True)
print(data.shape)

data_dir = "%s/data/%s/tmdata_complet_%d_%d_%s_%s_%s_s_select.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data.to_csv(data_dir, sep=",", index = False)

data.head()

In [ ]:
# Select sub Prague rectangles and sort by date - tm data

dir_data_tm = "%s/data/%s/tmdata_%d_%d_%s_%s_%s.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data_tm = pd.read_csv(dir_data_tm)

x_min = -743800
x_max = -735800
y_min = -1050600
y_max = -1042600

data_tm = data_tm.loc[lambda data: (data_tm.praha == 1) & (data_tm.x >= x_min) & (data_tm.x <= x_max) & (data_tm.y >= y_min) & (data_tm.y <= y_max), :]
data_tm.sort_values('date',inplace=True)
print(data_tm.shape)

data_dir = "%s/data/%s/tmdata_%d_%d_%s_%s_%s_s_select.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data_tm.to_csv(data_dir, sep=",", index = False)


data_tm.sort_values(['date'], ascending=[1], inplace=True)
print(data_tm.shape)

In [ ]:
# Select sub Prague rectangles and sort by date - neighbours data
dir_neigh = "%s/output/python/neighbour_arrays_%d_%s_%s_%s_x_%d_s.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data = pd.read_csv(dir_neigh)
print(data.shape)

x_min = -744800
x_max = -734800
y_min = -1051600
y_max = -1041600

data = data.loc[lambda data: (data.x >= x_min) & (data.x <= x_max) & (data.y >= y_min) & (data.y <= y_max), :]
data.sort_values('date',inplace=True)
print(data.shape)

dir_neigh = "%s/output/python/neighbour_arrays_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data.to_csv(dir_neigh, sep=",", index = False)

data.head()

In [ ]:
# Read neighbours data
data_dir = "%s/output/python/neighbour_arrays_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data_n = pd.read_csv(data_dir)
print(data_n.shape)
data_n.head()

In [ ]:
# Read complet tm data
data_dir = "%s/data/%s/tmdata_complet_%d_%d_%s_%s_%s_s_select.csv" % (main_dir, crime_type, days, raster, crime_type, date_from, date_to)
data = pd.read_csv(data_dir)
print(data.shape)
data_s = data.loc[lambda data: (data.praha == 1), :]
print(data_s.shape)
data_s.head()

In [ ]:
# Concat dataframes

result = pd.merge(data_s, data_n, how='left', on=['id', 'date'])
print(result.shape)
result.head()
data_dir = "%s/output/python/complet_tm_neigh_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
result.to_csv(data_dir, sep=",", index = False)

In [ ]:
data_dir = "%s/output/python/complet_tm_neigh_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data = pd.read_csv(data_dir)
print(data.shape)
data.head()

In [ ]:
# Feature Extraction with PCA

X_long_lat = ['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9',
       'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17', 'd18',
       'd19', 'd20', 'd21', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68',
       '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79',
       '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90',
       '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101',
       '102', '103', '104', '105', '106', '107', '108', '109', '110',
       '111', '112', '113', '114', '115', '116', '117', '118', '119',
       '120', 'Y', 'Mo', 'W', 'WD', 'D', 'MoCat', 'lokalitarelevance', 'vzdbcs',
       'vzdpostabanka', 'vzdprodejna', 'vzdrestaurace', 'stavobj_A', 'stavobj_B',
       'stavobj_C', 'stavobj_D', 'stavobj_F', 'stavobj_G', 'datodHCat_1',
       'datodHCat_2', 'datodHCat_3', 'datodHCat_4', 'datodHCat_5']

features = len(X_long_lat )

pca = PCA(n_components=features-1)
fit = pca.fit(data.loc[:,X_long_lat])

result =  pca.fit_transform(data.loc[:,X_long_lat])
result.shape

for i in range(result.shape[0]):
    name = 'PCA_%d'%i
    data[name] = result[:,i]

In [ ]:
data_dir = "%s/output/python/complet_tm_neigh_%d_%s_%s_%s_x_%d_s_select_pca_all.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data.to_csv(data_dir, sep=",", index = False)

In [ ]:
data_dir = "%s/output/python/complet_tm_neigh_%d_%s_%s_%s_x_%d_s_select_pca.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data = pd.read_csv(data_dir)
print(data.shape)
data.head()